# Week 3 Assignment Part 1#

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.
Start by creating a new Notebook for this assignment.
Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

## To create the dataframe ##
The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [71]:
# Load & Import Required Libraries
import pandas as pd
import requests
!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup

Solving environment: done

# All requested packages already installed.



In [72]:
# open Wiki page with Beautiful Soup

data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data, 'html.parser')

In [73]:
# create empty lists
postcodeL = []
boroughL = []
neighbourhoodL = []

# Parse scraped data into empty lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postcodeL.append(cells[0].text)
        boroughL.append(cells[1].text)
        neighbourhoodL.append(cells[2].text.rstrip('\n')) # clean up the "\n" form the neighborhood column

In [74]:
# Convert lists into pandas dataframe
tor_nhood = [('Postcode', postcodeL),
                      ('Borough', boroughL),
                      ('Neighborhood', neighbourhoodL)]
tor_df = pd.DataFrame.from_dict(dict(tor_nhood))
tor_df.head(10)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


In [75]:
# Remove rows where Borough is not assigned and reset index
tor_df_na = tor_df[tor_df.Borough != 'Not assigned'].reset_index(drop=True)
tor_df_na.head(10)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Not assigned
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


In [76]:
# Group Neighbourhoods with the same Postcode and Borough in the same row,
# separated by comma
tor_df_group = tor_df_na.groupby(['Postcode','Borough'], as_index=False).agg(lambda x: ','.join(x))
tor_df_group.head(10)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [77]:
# Change Neighborhood not assigned values
# to the corresponding borough name
nhood_rows = tor_df_group.Neighborhood == 'Not assigned'
tor_df_group.loc[nhood_rows, 'Neighborhood'] = tor_df_group.loc[nhood_rows, 'Borough']

In [83]:
# Rename clean dataframe and check shape
tor_df_clean = tor_df_group
tor_df_clean.shape

(103, 3)

In [84]:
# Save clean dataframe as csv file named 'can_post_df'
tor_df_clean.to_csv(r'can_post_df.csv')